In [1]:
import pandas as pd
import json
import neattext as nt
import neattext.functions as nfx
import matplotlib_inline
import re
import nltk
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

%matplotlib.inline

UsageError: Line magic function `%matplotlib.inline` not found.


In [2]:

file = pd.read_csv("movies_metadata.csv")

movies = pd.DataFrame(file[["imdb_id", "original_title", "overview", "genres"]])

new_genre = []
for i in range(0, len(movies["genres"])):
    new_genre.append([])
    for j in json.loads(movies["genres"][i].replace("\'", "\"")):
        new_genre[i].append(j["name"])

movies['genre_new'] = new_genre
movies = pd.DataFrame(movies)

/tmp/ipykernel_5233/4022294793.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv("movies_metadata.csv")


In [3]:
movies_new = movies[~(movies['genre_new'].str.len() == 0)]

In [4]:
def clean_txt(text):
    text = re.sub("[^a-zA-Z]", " ", str(text))
    text = text.lower()
    return text


In [5]:
movies_new["overview_clean"] = movies_new["overview"].apply(lambda x: clean_txt(x))

movies_new['overview_clean'].apply(lambda x: nt.TextExtractor(x).extract_stopwords())
movies_new['overview_clean'] = movies_new["overview_clean"].apply(nfx.remove_stopwords)


/tmp/ipykernel_5233/2082575070.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_new["overview_clean"] = movies_new["overview"].apply(lambda x: clean_txt(x))
/tmp/ipykernel_5233/2082575070.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_new['overview_clean'] = movies_new["overview_clean"].apply(nfx.remove_stopwords)


In [6]:
nltk.download('stopwords')

stop_words = set(stopwords.words("english"))


def remove_stopwords(text):
    no_stopwords_text = [w for w in text.split() if not w in stop_words]
    return ' '.join(no_stopwords_text)

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

def lemma(text):
    wordnet = WordNetLemmatizer()
    tokenization = nltk.word_tokenize(text)
    lemma_words = [wordnet.lemmatize(x) for x in tokenization]
    return ' '.join(lemma_words)


movies_new["overview_clean"] = movies_new["overview_clean"].apply(lambda x: lemma(x))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/prithvij/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/prithvij/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/prithvij/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/prithvij/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
/tmp/ipykernel_5233/3393935177.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_new["overview_clean"] = movies_new["overview_clean"].apply(lambda x: lemma(x))


In [7]:
movies_new.head()

,imdb_id,original_title,overview,genres,genre_new,overview_clean
0,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[Animation, Comedy, Family]",led woody andy toy live happily room andy birt...
1,tt0113497,Jumanji,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[Adventure, Fantasy, Family]",sibling judy peter discover enchanted board ga...
2,tt0113228,Grumpier Old Men,A family wedding reignites the ancient feud be...,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...","[Romance, Comedy]",family wedding reignites ancient feud door nei...
3,tt0114885,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...","[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...","[Comedy, Drama, Romance]",cheated mistreated stepped woman holding breat...
4,tt0113041,Father of the Bride Part II,Just when George Banks has recovered from his ...,"[{'id': 35, 'name': 'Comedy'}]",[Comedy],george bank recovered daughter wedding receive...


In [17]:
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(movies_new["genre_new"])

y = multilabel_binarizer.transform(movies_new["genre_new"])

tfidf = TfidfVectorizer(max_features=10000)

xtrain, xtest, ytrain, ytest = train_test_split(movies_new['overview_clean'], y, train_size=0.8)

xtrain_tfidf = tfidf.fit_transform(xtrain)
xtest_tfidf = tfidf.transform(xtest)

clf = BinaryRelevance(MultinomialNB())

clf.fit(xtrain_tfidf, ytrain)

y_pred_prob = clf.predict_proba(xtest_tfidf)


In [18]:

t = 0.2
y_pred_new = (y_pred_prob >= t).astype(int)
print(f1_score(ytest, y_pred_new, average="micro"))

0.5491325175606245


In [20]:
def infer_tags(q):
    q = clean_txt(q)
    q = remove_stopwords(q)
    q = lemma(q)
    q_vec = tfidf.transform([q])
    q_pred = clf.predict(q_vec)
    return multilabel_binarizer.inverse_transform(q_pred)


for i in range(5):
    k = xtest.sample(1).index[0]
    print("Movie: ", movies_new['original_title'][k], "\nPredicted genre: ", infer_tags(xtest[k])), print(
        "Actual genre: ", movies_new['genre_new'][k], "\n")


Movie:  21 nuits avec Pattie 
Predicted genre:  [('Comedy', 'Drama')]
Actual genre:  ['Comedy'] 

Movie:  El buque maldito 
Predicted genre:  [()]
Actual genre:  ['Horror'] 

Movie:  The Spiral Staircase 
Predicted genre:  [('Drama',)]
Actual genre:  ['Crime', 'Drama', 'Horror', 'Mystery', 'Thriller'] 

Movie:  Act Like You Love Me 
Predicted genre:  [('Comedy', 'Drama')]
Actual genre:  ['Comedy', 'Romance'] 

Movie:  Un amour de sorcière 
Predicted genre:  [()]
Actual genre:  ['Adventure', 'Comedy', 'Fantasy', 'Romance'] 

